# Creación de nuevos tipos de datos en Julia

## ¿Por qué necesitamos crear nuevos tipos?

En el notebook 1, empezamos a desarrollar la aritmética de intervalos. Para hacerlo, tratamos a un intervalo como un par ordenado de números de la forma $(a, b)$, representando el intervalo cerrado $[a, b]$. Pero un vil par de números ¡*puede representar muchos tipos de objetos diferentes*! --por ejemplo, un número complejo, un vector de dos componentes, o un "número dual" (ver más tarde en el curso).
Además, un intervalo no es únicamente un par ordenado de números, sino cuenta con otras propiedades que queremos capturar.

Para poder representar dentro de la computadora la gama de propiedades de un intervalo, y para distinguirlo de otros objetos que se representen superficialmente de la misma manera, requerimos codificar la definición de un intervalo en un *nuevo tipo de datos*, que llamaremos `Intervalo`.

## Despacho múltiple ("multiple dispatch")

Una característica clave de Julia, que lo distingue de la gran mayoría de los demás lenguajes, es el "despacho múltiple". Esto quiere decir que una misma función (llamada una *función genérica*) puede tener varios *métodos* que actúen sobre objetos de diferentes tipos.

Pensemos en el notebook 1. Podríamos querer definir la función `f` actuando sobre una variable para representar la función usual $f: \mathbb{R} \to \mathbb{R}$, pero cuando actúa sobre dos variables, tratamos a estos dos variables como representando un intervalo, y entonces aplicamos la extensión intervalar $\tilde{f}: \mathbb{IR} \to \mathbb{IR}$ que mapea un intervalo a su rango.

**[1]** (i) Implementa la función $f(x) = x^2 - 2x$ de las dos formas, con `f(x)` y `f(a, b)`.

(ii) ¿Qué arroja `methods(f)`?

(iii) Escribe un método de `f` tratando al intervalo como un par ordenado ("tupla" / "tuple"). Para hacerlo, al momento de definir la función, se incluye una *anotación de tipo* ("type annotation") `::Tuple` o `::NTuple{2}` (para especificar que debe tener dos entradas):

    f(x::Tuple) = ...
    
(iv) Ahora ¿qué arroja `methods(f)`?`

In [1]:
using Base.Test

Implementaremos la función $f(x) = x^2 - 2x$

In [2]:
function f(x)
    return x^2-2*x
end

f (generic function with 1 method)

Y ahora implementaremos la función que determina la imagen de la función $f(x) = x^2 -2x$ de un intervalo $(a,b)$, para ello el argumento de la función son los extremos del intervalo

In [3]:
function f(a, b)
    if a < 0 && b >0
        if abs(a) < abs(b)
            return (0, b^2)
        else
            return (0, a^2)
        end
    elseif a < 0 && b < 0
         return (b^2, a^2)
    else
        return (a^2, b^2)
    end
end

f (generic function with 2 methods)

(ii) ¿Qué arroja `methods(f)`?

In [4]:
methods(f)

# 2 methods for generic function "f":
f(x) in Main at In[2]:2
f(a, b) in Main at In[3]:2

Que la función `f` tiene dos métodos genericos, uno definido en la entrada $3$ y la otra en las $3$

* `In[2]:2`
* `In[3]:2`

(iii) Escribe un método de `f` tratando al intervalo como un par ordenado ("tupla" / "tuple"). Para hacerlo, al momento de definir la función, se incluye una *anotación de tipo* ("type annotation") `::Tuple` o `::NTuple{2}` (para especificar que debe tener dos entradas):

    f(x::Tuple) = ...
    

In [5]:
#f(x::NTuple{2}) = x.^2 .- 2 .* x # Def. la función f, cuyo argumento es una tupla
function f(x::NTuple{2})
    if x[1] < 0 && x[2] >0
        if abs(x[1]) < abs(x[2])
            return (0, x[2]^2)
        else
            return (0, x[1]^2)
        end
    elseif x[1] < 0 && x[2] < 0
         return (x[2]^2, x[1]^2)
    else
        return (x[1]^2, x[2]^2)
    end
end

f (generic function with 3 methods)

Observamos que ahora la función `f` está definida con tres métodos

In [6]:
methods(f)

# 3 methods for generic function "f":
f(x::Tuple{T,T} where T) in Main at In[5]:3
f(x) in Main at In[2]:2
f(a, b) in Main at In[3]:2

Efectivamente vemos que ahora `f` tiene tres métodos definidos, observamos que en la última definición de `f`, tiene como único argumento una tupla de dos elementos, por lo que al evaluar la función en una tupla de tres elementos `f` regresaria un error, es decir;

In [7]:
f((1,2,8))

LoadError: [91mMethodError: no method matching *(::Tuple{Int64,Int64,Int64}, ::Tuple{Int64,Int64,Int64})[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424[39m

In [8]:
typeof((2,3))

Tuple{Int64,Int64}

Aquí sólo revisamos que (2,3), es una `Tuple{Int64,Int64}`

Ahora revisemos que las definiciones de `f`, estén bien, para ello haremos algunos `test`

In [9]:
@testset "Test para f" begin
    @test f(2) == 0
    @test f(2,3) == (4,9)
    @test f((2,3)) == (4,9)
    #@test f((1,2,3)) == (-1,0,3)
end

Test Summary: | Pass  Total
Test para f   |    3      3


Base.Test.DefaultTestSet("Test para f", Any[], 3, false)

## Creación de tipos

Julia nos permite crear / definir nuevos *tipos de datos*, o *tipos* ("types") para representar objetos en el programa que se comporten de cierta forma, por ejemplo para modelar un objeto en el mundo real, o un nuevo tipo de número. En Julia (a diferencia de muchos otros lenguajes), los tipos que definamos los usuarios tienen *exactamente el mismo "nivel" que los tipos pre-definidos*.

La definición de un tipo para representar un intervalo se ve así:

In [10]:
struct Intervalo  # Definimos la estructura intervalo
    inf::Float64  # Definimos inf como un Float64
    sup::Float64  # De igual manera para sup 
end

Esto define un nuevo tipo de objeto que se llama `Intervalo`, y especifica los datos que contiene: dos `Float64`, con los nombre `inf` (cota inferior) y `sup` (cota superior). Es decir, podemos pensar que un objeto de tipo `Intervalo` corresponde a una "caja" con estos datos adentro. Podremos crear distintos objetos que tienen el mismo *tipo*, pero que difieren entre sí; corresponde a crear distintas cajas que se ven superficialmente iguales, pero tienen cosas diferentes adentro.

[Nota: Las definiciones de tipos no se pueden modificar dentro de una misma sesión de Julia.]

### Constructores


Hasta ahora, no se ha creado ningún objeto de este tipo; solo hemos especificado una plantilla para cómo se ven todo `Intervalo`. Para poder crear un objeto de este tipo, Julia nos provee, de forma automática (por el momento), una función con el mismo nombre que el tipo; esta función se llama el *constructor* del tipo.

**[2]** (i) Encuentra cómo utilizar el constructor para crear un intervalo $X = [3, 4]$.

(ii) Verifica que puedes extraer la información adentro del objeto utilizando `X.<algo>`. [Pista: Para ver las opciones, presione `TAB` después del `.`.]

(iii) ¿Se puede modificar los datos? ¿Por qué?

(iv) ¿Qué tipos se pueden utilizar como argumentos al constructor?

(v) La función `parse` se utiliza para convertir una cadena en un número. Utiliza la documentación de `parse` (el cual se obtiene con `?parse`) para escribir un nuevo *método* (versión) del constructor que acepta dos cadenas. [Pista: Se especifica que un argumento debe ser de tipo `String` con `::String`.]

(vi) Escribe unos tests sencillos del constructor de `Intervalo`. Utiliza `@test_throws` para verificar los errores que pueda arrojar el constructor.

(i) Encuentra cómo utilizar el constructor para crear un intervalo $X = [3, 4]$. Para ello llamamos el constructor `Intervalo` a un intervalo, es decir;

In [11]:
x = Intervalo(3,4)

Intervalo(3.0, 4.0)

Veamos que la salida de `Intervalo` si es un intervalo

In [12]:
typeof(Intervalo(3,4))

Intervalo

Vemos que sí, la salida de `Intervalo` si en un intervalo

(ii) Verifica que puedes extraer la información adentro del objeto utilizando `X.<algo>`. [Pista: Para ver las opciones, presione `TAB` después del `.`.]

In [13]:
x.inf

3.0

Vemos que para extraer la información del objeto se utiliza `x.inf` tal como se definió en el constructor, entonces para el sup se llama como `x.sup`

In [14]:
x.sup

4.0

In [15]:
a , b = Int16(2), Int16(3)

(2, 3)

In [16]:
Intervalo(a,b)

Intervalo(2.0, 3.0)

In [17]:
Intervalo([1,2],3)

LoadError: [91mMethodError: Cannot `convert` an object of type Array{Int64,1} to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.[39m

Vemos que `Intervalo` no acept como argumento una arreglo

Cualquier  `Int` y `Float`

(iii) ¿Se puede modificar los datos?

Intentemos modificar el constructor y ver si se puede mofificar o no, para ello modifiquemos `inf` como un `Array{Int64,1}` 

In [18]:
struct Intervalo  # Definimos la estructura intervalo
    inf::Array{Int64,1}  # Definimos inf como un Float64
    sup::Float64  # De igual manera para sup 
end

LoadError: [91minvalid redefinition of constant Intervalo[39m

Vemos que es inválida la refinición del constructor

In [19]:
?struct

search: struct mutable struct unsafe_trunc



The most commonly used kind of type in Julia is a struct, specified as a name and a set of fields.

```
struct Point
    x
    y
end
```

Fields can have type restrictions, which may be parameterized:

```
struct Point{X}
    x::X
    y::Float64
end
```

A struct can also declare an abstract super type via `<:` syntax:

```
struct Point <: AbstractPoint
    ...
```

Structs are immutable by default; an instance of one of these types cannot be modified after construction. Use `mutable struct` instead to declare a type whose instances can be modified.

See the manual for more details, such as how to define constructors.


Vemos que por defecto los constructores son inmutables y para hacerlos
mutables debemos usar `mutable strct`

(iv) ¿Qué tipos se pueden utilizar como argumentos al constructor?

Alparecer no hay restricciones, incluso se pueden declarar tipo abstractos, `AbstractPoint`

(v) La función `parse` se utiliza para convertir una cadena en un número. Utiliza la documentación de `parse` (el cual se obtiene con `?parse`) para escribir un nuevo *método* (versión) del constructor que acepta dos cadenas. [Pista: Se especifica que un argumento debe ser de tipo `String` con `::String`.]

In [20]:
?parse

search: parse ParseError sparse sparsevec SparseVector SparseArrays



```
parse(str, start; greedy=true, raise=true)
```

Parse the expression string and return an expression (which could later be passed to eval for execution). `start` is the index of the first character to start parsing. If `greedy` is `true` (default), `parse` will try to consume as much input as it can; otherwise, it will stop as soon as it has parsed a valid expression. Incomplete but otherwise syntactically valid expressions will return `Expr(:incomplete, "(error message)")`. If `raise` is `true` (default), syntax errors other than incomplete expressions will raise an error. If `raise` is `false`, `parse` will return an expression that will raise an error upon evaluation.

```jldoctest
julia> parse("x = 3, y = 5", 7)
(:(y = 5), 13)

julia> parse("x = 3, y = 5", 5)
(:((3, y) = 5), 13)
```

```
parse(str; raise=true)
```

Parse the expression string greedily, returning a single expression. An error is thrown if there are additional characters after the first expression. If `raise` is `true` (default), syntax errors will raise an error; otherwise, `parse` will return an expression that will raise an error upon evaluation.

```jldoctest
julia> parse("x = 3")
:(x = 3)

julia> parse("x = ")
:($(Expr(:incomplete, "incomplete: premature end of input")))

julia> parse("1.0.2")
ERROR: ParseError("invalid numeric constant \"1.0.\"")
Stacktrace:
[...]

julia> parse("1.0.2"; raise = false)
:($(Expr(:error, "invalid numeric constant \"1.0.\"")))
```

```
parse(type, str, [base])
```

Parse a string as a number. If the type is an integer type, then a base can be specified (the default is 10). If the type is a floating point type, the string is parsed as a decimal floating point number. If the string does not contain a valid number, an error is raised.

```jldoctest
julia> parse(Int, "1234")
1234

julia> parse(Int, "1234", 5)
194

julia> parse(Int, "afc", 16)
2812

julia> parse(Float64, "1.2e-3")
0.0012
```


Una vez revisada la documentación de `parse`, defineremos un nuevo método de `Intervalo` que acepte dos cadenas

In [21]:
function Intervalo(inf::String,sup::String)
    Intervalo(parse(Float64, inf), parse(Float64, sup)) 

end

Intervalo

(vi) Escribe unos tests sencillos del constructor de `Intervalo`. Utiliza `@test_throws` para verificar los errores que pueda arrojar el constructor.

In [22]:
@testset "Test Intervalo" begin
    @test Intervalo(5.2,π) == Intervalo(5.2, 3.141592653589793)
    @test Intervalo("3.14159", "4.0e-4") == Intervalo(3.14159, 0.0004)
    @test Intervalo("3.14159","-0") == Intervalo(3.14159, -0.0)
end

Test Summary:  | Pass  Total
Test Intervalo |    3      3


Base.Test.DefaultTestSet("Test Intervalo", Any[], 3, false)

Veamos cuántos métodos tiene definido `Intervalo`

In [23]:
methods(Intervalo)

# 4 methods for generic function "(::Type)":
Intervalo(inf::Float64, sup::Float64) in Main at In[10]:2
Intervalo(inf::String, sup::String) in Main at In[21]:2
Intervalo(inf, sup) in Main at In[10]:2
(::Type{T})(arg) where T in Base at sysimg.jl:77

vemos que se tienen cuatro métodos definidos

In [24]:
?@test_throws

```
@test_throws extype ex
```

Tests that the expression `ex` throws an exception of type `extype`. Note that `@test_throws` does not support a trailing keyword form.


In [25]:
@test_throws Intervalo

LoadError: [91mMethodError: no method matching @test_throws(::Symbol)[0m
Closest candidates are:
  @test_throws(::ANY, [91m::ANY[39m) at test.jl:376[39m

## Sobrecarga de operadores

Ahora sabemos cómo definir una variables $X = [3, 4]$ y $Y = [5, 6]$. Pero las operaciones aritméticas, como las vimos en el notebook 1, son torpes: para dos intervalos debemos escribir `suma(X, Y)` en lugar de `X + Y`.

**[3]** Define la función `suma` para que funcione únicamente para dos `Intervalo`s, usando la misma regla que en el notebook 1.

In [26]:
function suma(x::Intervalo, y::Intervalo)
    return Intervalo(x.inf + y.inf, x.sup + y.sup) 
    # En la linea anterior suma entrada a entrada cada Intervalo
end

suma (generic function with 1 method)

Julia, como la mayoría de los lenguajes modernos, permite utilizar la *sobrecarga de operadores* para extender la definición de `+` a nuestro nuevo tipo.

Para extender una función que viene definida en `Base` (la parte básica del lenguaje), como lo es `+`, se tiene que importar:

In [27]:
import Base: +, -,*,/

**[4]** (i) Examina la lista de *métodos* (versiones) que ya existen de la función `+` usando la función `methods`.

(ii) Agrega un método nuevo que sume dos intervalos. [Pista: Basta con definir `+` con dos argumentos especificados como de tipo `Intervalo`, como si fuera una función nueva.]

(iii) Agrega un método que sume un intervalo y un número real. El número se puede especificar con el tipo abstracto `Real`.

(iv) Agrega un método que sume un número real y un intervalo en el otro orden.

(v) Escribe tests para estos métodos.

**[4]** (i) Examina la lista de *métodos* (versiones) que ya existen de la función `+` usando la función `methods`.

In [28]:
methods(+)

# 180 methods for generic function "+":
+(x::Bool, z::Complex{Bool}) in Base at complex.jl:232
+(x::Bool, y::Bool) in Base at bool.jl:89
+(x::Bool) in Base at bool.jl:86
+(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:96
+(x::Bool, z::Complex) in Base at complex.jl:239
+(a::Float16, b::Float16) in Base at float.jl:372
+(x::Float32, y::Float32) in Base at float.jl:374
+(x::Float64, y::Float64) in Base at float.jl:375
+(z::Complex{Bool}, x::Bool) in Base at complex.jl:233
+(z::Complex{Bool}, x::Real) in Base at complex.jl:247
+(x::Char, y::Integer) in Base at char.jl:40
+(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:312
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:334
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:327
+(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:321
+(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:289
+(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:346
+(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:362
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:460
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:453
+(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:447
+(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:308
+(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:277
+(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:284
+(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:292
+(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:300
+(B::BitArray{2}, J::UniformScaling) in Base.LinAlg at linalg/uniformscaling.jl:59
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem, b::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:87
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuild, b::Base.Pkg.Resolve.VersionWeights.VWPreBuild) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:135
+(a::Base.Pkg.Resolve.VersionWeights.VersionWeight, b::Base.Pkg.Resolve.VersionWeights.VersionWeight) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:197
+(a::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue, b::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue) in Base.Pkg.Resolve.MaxSum.FieldValues at pkg/resolve/fieldvalue.jl:44
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.CompoundPeriod) in Base.Dates at dates/periods.jl:349
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.Period) in Base.Dates at dates/periods.jl:347
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.TimeType) in Base.Dates at dates/periods.jl:387
+(x::Date, y::Base.Dates.Day) in Base.Dates at dates/arithmetic.jl:77
+(x::Date, y::Base.Dates.Week) in Base.Dates at dates/arithmetic.jl:75
+(dt::Date, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:58
+(dt::Date, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:32
+(dt::Date, t::Base.Dates.Time) in Base.Dates at dates/arithmetic.jl:20
+(t::Base.Dates.Time, dt::Date) in Base.Dates at dates/arithmetic.jl:24
+(x::Base.Dates.Time, y::Base.Dates.TimePeriod) in Base.Dates at dates/arithmetic.jl:81
+(dt::DateTime, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:52
+(dt::DateTime, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:28
+(x::DateTime, y::Base.Dates.Period) in Base.Dates at dates/arithmetic.jl:79
+(y::AbstractFloat, x::Bool) in Base at bool.jl:98
+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:32
+(x::Integer, y::Ptr) in Base at pointer.jl:128
+(z::Complex, w::Complex) in Base at complex.jl:221
+(z::Complex, x::Bool) in Base at complex.jl:240
+(x::Real, z::Complex{Bool}) in Base at complex.jl:246
+(x::Real, z::Complex) in Base at complex.jl:258
+(z::Complex, x::Real) in Base at complex.jl:259
+(x::Rational, y::Rational) in Base at rational.jl:245

(ii) Agrega un método nuevo que sume dos intervalos. [Pista: Basta con definir `+` con dos argumentos especificados como de tipo `Intervalo`, como si fuera una función nueva.]

In [29]:
function +(x::Intervalo, y::Intervalo)
    return Intervalo(x.inf + y.inf, x.sup + y.sup)
end

+ (generic function with 181 methods)

(iii) Agrega un método que sume un intervalo y un número real. El número se puede especificar con el tipo abstracto `Real`.

(iv) Agrega un método que sume un número real y un intervalo en el otro orden.

In [30]:
function +(x::Intervalo, c::Real) 
    # Aquí definimos la suma de un Intervalo a un Real
    return Intervalo(x.inf + c, x.sup + c)
end

function +(c::Real, x::Intervalo)
    # Aquí definimos la suma de un Real a un Intervalo
    return Intervalo(x.inf + c, x.sup + c)
end

+ (generic function with 183 methods)

(v) Escribe tests para estos métodos.

In [31]:
@testset "Tests de +" begin
    @test 3 + Intervalo(0,0) == Intervalo(3, 3)
    @test Intervalo(3,3) + Intervalo(0,0) == Intervalo(3, 3)
    @test Intervalo(3.0,3) + Intervalo("4","3.14") == Intervalo(3 + 4, 3 + 3.14)
    @test Intervalo(3.0,3) + Intervalo(4,π) == Intervalo(3 + 4, 3 + pi)
    @test Intervalo(0,0) + 3 == Intervalo(3,3)
    
end

Test Summary: | Pass  Total
Tests de +    |    5      5


Base.Test.DefaultTestSet("Tests de +", Any[], 5, false)

**[5]** Escribe las funciones `-` y `*`, así como tests para ellos.

In [32]:
function -(x::Intervalo, y::Intervalo)
    return Intervalo(x.inf - y.inf, x.sup - y.sup)
end

function -(x::Intervalo, c::Real)
    return Intervalo(x.inf - c, x.sup - c)
end

function -(c::Real, x::Intervalo)
    return Intervalo(c-x.inf, c-x.sup)
end

- (generic function with 197 methods)

In [33]:
function *(x::Intervalo, y::Intervalo)
    Intervalo(min(x.inf*y.inf, x.sup*y.inf, x.sup*y.sup, x.inf*y.sup), 
        max(x.inf*y.inf, x.sup*y.inf, x.sup*y.sup, x.inf*y.sup))
end

function *(x::Intervalo, c::Real)
    Intervalo(min(x.inf*c, x.sup*c), max(x.inf*c, x.sup*c))
    
    #return Intervalo(x.inf * c, x.sup * c)
end

function *(c::Real, x::Intervalo)
    Intervalo(min(x.inf*c, x.sup*c), max(x.inf*c, x.sup*c))
end

* (generic function with 185 methods)

**[6]** (i) Escribe un método de `/` para dividir un intervalo por un número. ¿Cuáles son los casos especiales?

(ii) Para dividir dos intervalos `X` y `Y`, hay un caso particular. ¿Cuál es? ¿Cuál debe ser el resultado?

(iii) Escribe un método para la función `inv` que calcula la inversa de un intervalo `x`, es decir `1 / x`.

(iv) Utiliza la función `inv` para definir `X / Y` en general para dos intervalos `X` y `Y`.

(v) Escribe tests para `/`.

(i) Escribe un método de `/` para dividir un intervalo por un número. ¿Cuáles son los casos especiales?

Al parecer los casos especiales son:

* Algún elemento del Intervalo en el denominador es cero
* 

In [34]:
function /(x::Intervalo, y::Intervalo)
    Intervalo(min(x.inf/y.inf, x.inf/y.sup, x.sup/y.inf, x.sup/y.sup), 
        max(x.inf/y.inf, x.sup/y.inf, x.sup/y.sup, x.inf/y.sup))
end

function /(x::Intervalo, c::Real)
    Intervalo(min(x.inf/c, x.sup/c), max(x.inf/c, x.sup/c))
    
    #return Intervalo(x.inf * c, x.sup * c)
end

function /(c::Real, x::Intervalo)
    Intervalo(min(c/x.inf, c/x.sup), max(c/x.inf, c/x.sup))
end

/ (generic function with 76 methods)

In [35]:
Intervalo(0,1/0)/Intervalo(13,1/0) == Intervalo(NaN, NaN)

false

In [36]:
NaN == NaN

false

- Vemos que `NaN` no es igual a `Nan`, por lo que el caso un especial es: `Intervalo(0,1/0)/Intervalo(13,1/0) != Intervalo(NaN, NaN)`  

(ii) Para dividir dos intervalos `X` y `Y`, hay un caso particular. ¿Cuál es? ¿Cuál debe ser el resultado?

In [37]:
Intervalo(3,4)/Intervalo(3,4)

Intervalo(0.75, 1.3333333333333333)

El caso especial es dividir un intervalo por sí mismo, cuyo resulado debería ser el intervalo (1,1), pero como en el caso anterior y por como se definió la división de intervalos no regresa el intervalo (1,1)

(iii) Escribe un método para la función `inv` que calcula la inversa de un intervalo `x`, es decir `1 / x`.

In [38]:
inverso(x::Intervalo) = 1/x
inverso(x::Intervalo,y::Intervalo) = y/x

inverso (generic function with 2 methods)

(v) Escribe tests para `/`.

In [39]:
@testset "test para /" begin
    #@test Intervalo(0,1/0)/Intervalo(13,1/0) == Intervalo(NaN,NaN)
    @test Intervalo(3,10)/Intervalo(3,5) == Intervalo(3/5,10/3)
    @test Intervalo(0,10)/Intervalo(-3,5) == Intervalo(10/-3,10/5)
    @test Intervalo(-3,0)/Intervalo(-3,4) == Intervalo(-3/4,1)
    @test Intervalo(-3,0)/3 == Intervalo(-1,0)
    @test Intervalo(-3,0)/(-3) == Intervalo(-0.0,1)
    @test 5/Intervalo(1,2) == Intervalo(2.5, 5.0)
    @test Intervalo(1,2)*inverso(Intervalo(1,2)) == Intervalo(1/2,2)
    @test inverso(Intervalo(1,2),Intervalo(0,1)) == Intervalo(0,1)
    @test Intervalo(1,1)/Intervalo(1,1) == Intervalo(1,1)
    @test inverso(Intervalo(1,3)) == Intervalo(1/3,1)
end

Test Summary: | Pass  Total
test para /   |   10     10


Base.Test.DefaultTestSet("test para /", Any[], 10, false)

In [40]:
inverso(Intervalo(1,3))

Intervalo(0.3333333333333333, 1.0)

In [41]:
Intervalo(1,3)*inverso(Intervalo(1,3))

Intervalo(0.3333333333333333, 3.0)

De nuevo vemos que el calculo anterior en resultado debe ser el intervalo (1,1)

### Constructores internos

Podríamos decidir que es conveniente restringir la definición de intervalos $[a, b]$ a la situación en la cual $a \le b$. Por el momento no podemos hacer esto.

Para lograrlo, debemos "interferir" con el proceso de construcción de un tipo, permitiendo que se construya sólo si se cumpla la condición que requeramos. Para hacerlo, Julia provee *constructores internos* ("inner constructors").

**[7]** Busca documentación sobre constructores internos y utilízalos para impedir que se crean `Intervalo`s con $a > b$. 

In [42]:
struct NIntervalo
    inf::Float64
    sup::Float64
    NIntervalo(inf,sup) = inf > sup ? error("No es un intervalo valido") : new(inf,sup)
end

In [43]:
NIntervalo(3,6)

NIntervalo(3.0, 6.0)

**[8]** ¿Qué interpretación podría tener un interval $[a, b]$ con $a > b$? (Hay distintas respuestas posibles.)

* No es un intervalo válido, pero podríamos interpretarlo como  un intervalo en el sentido "opuesto"

## Resumen

**[9]** Escribe un resumen de lo que hayamos visto en este notebook.

En este notebook revisamos:

* La definición multiple de funciones, es decir a una función se le pueden definir diferentes métodos.

* También pudimos definir nuestros propios tipos, usando construtures los cuales por defecto son inmutables. Y a estos les podemos asignar el carecter de mutable. Los tipos que definimos los llamados `Intervalo` ya que con estos podremos implemetar las operaciones apropiadas para la aritmérica de intervalos.

* Julia permite la sobrecarga de operadores aritméticos, con lo cual sobrecargamos dichos operadores para de definir las operaciones para nuestros tipos (`Intervalo`) que corresponden a la aritmetica de intervalos.

* Por último implementamos constructores internos a nuestro tipos, por ejemplo imponer la condición $a<b$ para un intervalo.

* También en el camino revisamos la función `parse` que practicamente sirve para convertir números en forma de cadena en números legibles para la máquina.